In [2]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import random
import csv
import json
import pandas as pd
import requests
import re
print("Libraries imported successfully")

Libraries imported successfully


In [3]:
# Spoofer code
try:

    import sys
    import os

    from fp.fp import FreeProxy
    from fake_useragent import UserAgent
    from bs4 import BeautifulSoup
    from selenium import webdriver
    from selenium.webdriver import Chrome
    from selenium.webdriver.common.keys import Keys
    from selenium.webdriver.common.by import By
    from selenium.webdriver.chrome.options import Options
    from selenium.webdriver.support.ui import WebDriverWait
    from selenium.webdriver.common.action_chains import ActionChains
    from selenium.webdriver.support import expected_conditions as EC
    from selenium.common.exceptions import TimeoutException
    import time
    print('all module are loaded ')

except Exception as e:

    print("Error ->>>: {} ".format(e))


class Spoofer(object):

    def __init__(self, country_id=['US'], rand=True, anonym=True):
        self.country_id = country_id
        self.rand = rand
        self.anonym = anonym
        self.userAgent, self.ip = self.get()

    def get(self):
        ua = UserAgent()
        proxy = FreeProxy(country_id=self.country_id, rand=self.rand, anonym=self.anonym).get()
        ip = proxy.split("://")[1]
        return ua.random, ip


class DriverOptions(object):

    def __init__(self):

        self.options = Options()
        self.options.add_argument('--no-sandbox')
        self.options.add_argument('--start-maximized')
        self.options.add_argument('--start-fullscreen')
        self.options.add_argument('--single-process')
        self.options.add_argument('--disable-dev-shm-usage')
        self.options.add_argument("--incognito")
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_argument('--disable-blink-features=AutomationControlled')
        self.options.add_experimental_option('useAutomationExtension', False)
        self.options.add_experimental_option("excludeSwitches", ["enable-automation"])
        self.options.add_argument("disable-infobars")

        self.helperSpoofer = Spoofer()

        self.options.add_argument('user-agent={}'.format(self.helperSpoofer.userAgent))
        self.options.add_argument('--proxy-server=%s' % self.helperSpoofer.ip)


class WebDriver(DriverOptions):

    def __init__(self, path=''):
        DriverOptions.__init__(self)
        self.driver_instance = self.get_driver()

    def get_driver(self):

        print("""
        IP:{}
        UserAgent: {}
        """.format(self.helperSpoofer.ip, self.helperSpoofer.userAgent))

        PROXY = self.helperSpoofer.ip
        webdriver.DesiredCapabilities.CHROME['proxy'] = {
            "httpProxy":PROXY,
            "ftpProxy":PROXY,
            "sslProxy":PROXY,
            "noProxy":None,
            "proxyType":"MANUAL",
            "autodetect":False
        }
        webdriver.DesiredCapabilities.CHROME['acceptSslCerts'] = True

        path = os.path.join(os.getcwd(), '/chromedriver.exe')

        driver = webdriver.Chrome()
        driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
        driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source":
                "const newProto = navigator.__proto__;"
                "delete newProto.webdriver;"
                "navigator.__proto__ = newProto;"
        })

        return driver

all module are loaded 


In [4]:
# Function to create a csv by giving the field name, its contents, the directory name and filename

def create_csv(fields, rows, dirname, filename):
    with open(dirname+filename, 'w', encoding='UTF8', newline='') as csvfile: 
        # creating a csv writer object
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(fields) 
        csvwriter.writerows(rows)
    print(f"Saved csv with name {filename}")

In [9]:
# Create anti bot detection web driver

while True:
    try:
        driver = WebDriver()
        break
    except: # If Driver creation fails try again
        print("Driver creation failed.....Trying again")
        continue
print("Driver Successfully created")


        IP:103.73.194.2:80
        UserAgent: Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.90 Safari/537.36
        
Driver Successfully created


In [11]:
# Create Driver instance and open the webpage through selenium

driverinstance = driver.driver_instance
url = "https://www.tripadvisor.in/Restaurants-g297667-Jaisalmer_Jaisalmer_District_Rajasthan.html"
driverinstance.get(url)
time.sleep(3)

# Remove default filters to get all resutls
try:
    driverinstance.find_element_by_xpath(".//*[contains(text(), 'Clear all filters')]").click()
    time.sleep(10)
except:
    print("Could not find button")

restaurant_urls = []
# Using loop to get all the links from paginated list
for i in range(10):
    sleep_time = 5+5*random.random()
    time.sleep(10)
    soup = str(BeautifulSoup(driverinstance.page_source, 'html.parser'))
    soup = BeautifulSoup(soup, 'lxml')

    restaurants = soup.find_all('div', class_ = "cauvp Gi o")
    for res in restaurants:
        url = "https://www.tripadvisor.in" + res.find('a', class_ = "dMdkg _S fkFuK Vt u Gi").attrs['href']
        restaurant_urls.append(url)

    driverinstance.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        button = driverinstance.find_element_by_xpath(".//*[contains(text(), 'Next')]").click()
    except:
        print("Button not Found, ending search")
        break

print(len(restaurant_urls))

Button not Found, ending search
267


In [12]:
# Save all the links in a text file

with open("Project-Dataset/jaisalmer_restaurant_urls.txt", 'w') as output:
    for row in restaurant_urls:
        output.write(str(row) + '\n')
print("Attraction links saved in text file successfully")

Attraction links saved in text file successfully
